In [1]:
# Various setup, probably mostly unused

from env import TestEnv
import pandas as pd
import json
import os
from conf import LisaLogging
from bart.common.Utils import area_under_curve
from trappy.plotter import plot_trace
from IPython.display import display
from trappy import ILinePlot
from trappy.stats.grammar import Parser
from bart.sched.SchedMultiAssert import SchedMultiAssert
import pandas as pd
from trace import Trace
LisaLogging.setup()
import logging
logging.getLogger('Trace').setLevel(logging.ERROR)
logging.getLogger('Analysis').setLevel(logging.WARNING)
logging.getLogger('EnergyMeter').setLevel(logging.DEBUG)
%matplotlib inline
from platforms.juno_energy import juno_energy
from platforms.pixel_energy import pixel_energy
import tests.eas.generic
from tests.eas.generic import EnergyModelTest
import numpy as np
from IPython.display import display
from wlgen import RTA, Periodic
from executor import Executor

2017-02-06 16:53:32,294 INFO    : root         : Using LISA logging configuration:
2017-02-06 16:53:32,295 INFO    : root         :   /home/brejac01/sources/lisa/logging.conf


In [2]:
# Some nonsense to get caiman to work on Brendan's computer
p = os.getenv('PATH').split(':')
caiman_path = '/opt/ds5_v5.23.0/bin'
if caiman_path not in p:
    p.insert(0, caiman_path)
    os.environ['PATH'] = ':'.join(p)

In [3]:
from platforms.juno_energy import juno_energy

In [4]:
from platforms.hikey_energy import hikey_energy

In [5]:
te = TestEnv(test_conf={'modules': ['cgroups'], 'ftrace': {'events': ['sched_switch']}}, force_new=True)

2017-02-06 16:53:32,368 INFO    : TestEnv      : Using base path: /home/brejac01/sources/lisa
2017-02-06 16:53:32,369 INFO    : TestEnv      : Loading default (file) target configuration
2017-02-06 16:53:32,370 INFO    : TestEnv      : Loading target configuration [/home/brejac01/sources/lisa/target.config]...
2017-02-06 16:53:32,373 INFO    : TestEnv      : Loading custom (inline) test configuration
2017-02-06 16:53:32,374 INFO    : TestEnv      : External tools using:
2017-02-06 16:53:32,375 INFO    : TestEnv      :    ANDROID_HOME: /work/android-sdk-linux
2017-02-06 16:53:32,376 INFO    : TestEnv      :    CATAPULT_HOME: /home/brejac01/sources/lisa/tools/catapult
2017-02-06 16:53:32,377 INFO    : TestEnv      : Devlib modules to load: ['cpuidle', 'cgroups', 'cpufreq']
2017-02-06 16:53:32,379 INFO    : TestEnv      : Connecting Android target [04D314DF000BC57F]
2017-02-06 16:53:32,380 INFO    : TestEnv      : Connection settings:
2017-02-06 16:53:32,381 INFO    : TestEnv      :    {'

In [6]:
# Some nonsense to make LISA treat HiKey with the respect it deserves
from trappy.stats.Topology import Topology
hikey_topology = Topology(clusters=[[0, 1, 2, 3], [4, 5, 6, 7]])
te.topology = hikey_topology
te.platform['clusters'] = {'big': [0, 1, 2, 3], 'little': [4, 5, 6, 7]}

In [7]:
def spread_evenly(ntasks, ncpus):
    ret = [0 for _ in range(ncpus)]
    for i in range(ntasks):
        ret[i % ncpus] += 1
    return ret

def get_various_packing_placements(ntasks, ncpus):
    max_spread = spread_evenly(ntasks, ncpus)
    
    max_pack = [ntasks] + [0] * (ncpus - 1)
    
    half_pack = [ntasks / 2] + spread_evenly(ntasks - (ntasks / 2), ncpus -1)
    
    return [max_spread, half_pack, max_pack]

In [8]:
def get_everything(nrg_model, cpu=0, task_pct=4):
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    ncpus = len(cluster_node.cpus)
    min_cap = cpu_node.min_capacity
    task_cap = int(1024 * (task_pct / 100.))
    
    tasks_per_cpu = int(min_cap / task_cap)
    ntasks = tasks_per_cpu * ncpus
    task_distribs = get_various_packing_placements(ntasks, ncpus)
    
    util_distribs = []
    wloads = []
    for task_distrib in task_distribs:
        util_distrib = [0 for _ in range(len(nrg_model.cpus))]
        for i, num_tasks_on_cpu in enumerate(task_distrib):
            util_distrib[cluster_node.cpus[i]] = num_tasks_on_cpu * task_cap
        util_distribs.append(util_distrib)

    # Stagger the workloads so that they prevent shared idle states
    period_s = 10e-3
    stagger_s = 0.7e-3
    delays = np.arange(period_s, step=stagger_s)
    
    if len(delays) > ntasks:
        print "WARNING: not enough tasks to cover period with wakeups"
    
    tasks = {}
    for i in range(ntasks):
        tasks['{}pct_{}'.format(task_pct, i)] = {
            'kind': 'Periodic',
            'params': {
                'duty_cycle_pct': task_pct,
                'period_ms' : period_s * 1e3,
                'delay_s' : delays[i % len(delays)],
                'duration_s': 2
            }
        }
    
    wlspec = {
        'type': 'rt-app',
        'conf': {
            'class': 'profile',
            'params': tasks
        }
    }

    return task_distribs, util_distribs, wlspec

In [9]:
task_distribs, util_distribs, wlspec = get_everything(hikey_energy)

In [10]:
executor = Executor(te, {
        'confs': [{
                'tag': 'myconf',
                'flags': ['ftrace', 'freeze_userspace'],
            }],
        'wloads': {'mywl': wlspec},
        'iterations': 10
    })

2017-02-06 16:53:36,813 INFO    : Executor     : Loading custom (inline) test configuration
2017-02-06 16:53:36,815 INFO    : Executor     : 
2017-02-06 16:53:36,816 INFO    : Executor     : ################################################################################
2017-02-06 16:53:36,817 INFO    : Executor     : Experiments configuration
2017-02-06 16:53:36,818 INFO    : Executor     : ################################################################################
2017-02-06 16:53:36,819 INFO    : Executor     : Configured to run:
2017-02-06 16:53:36,820 INFO    : Executor     :      1 target configurations:
2017-02-06 16:53:36,821 INFO    : Executor     :       myconf
2017-02-06 16:53:36,822 INFO    : Executor     :      1 workloads (10 iterations each)
2017-02-06 16:53:36,824 INFO    : Executor     :       mywl
2017-02-06 16:53:36,825 INFO    : Executor     : Total: 10 experiments
2017-02-06 16:53:36,826 INFO    : Executor     : Results will be collected under:
2017-02-06 16:

In [11]:
executor.run()

2017-02-06 16:53:37,171 INFO    : Executor     : 
2017-02-06 16:53:37,173 INFO    : Executor     : ################################################################################
2017-02-06 16:53:37,174 INFO    : Executor     : Experiments execution
2017-02-06 16:53:37,175 INFO    : Executor     : ################################################################################
2017-02-06 16:53:37,176 INFO    : Executor     : 
2017-02-06 16:53:37,177 INFO    : Executor     : ================================================================================
2017-02-06 16:53:37,179 INFO    : Executor     : configuring target for [myconf] experiments
2017-02-06 16:53:37,323 WARNING : Executor     : Setting target SELinux in permissive mode
2017-02-06 16:53:37,396 WARNING : Executor     : cpufreq governor not specified, using currently configured governor
2017-02-06 16:53:37,398 INFO    : Workload     : Setup new workload mywl
2017-02-06 16:53:37,400 INFO    : Workload     : Workload duratio

2017-02-06 16:53:45,879 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000703
2017-02-06 16:53:48,337 INFO    : Executor     : Collected FTrace binary trace:
2017-02-06 16:53:48,339 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/1/trace.dat
2017-02-06 16:53:48,340 INFO    : Executor     : Collected FTrace function profiling:
2017-02-06 16:53:48,341 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/1/trace_stat.json
2017-02-06 16:53:48,342 INFO    : Executor     : Un-freezing userspace tasks
2017-02-06 16:53:48,672 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-06 16:53:48,674 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-06 16:53:48,676 INFO    : Executor     : Experiment 1/10, [myconf:mywl] 2/10
2017-02-06 16:53:48,719 INFO    : Executor     : Freezing all tasks except: sh,adbd,init,usb,transport,thermal-engine
2017-02-06 16:53:50

2017-02-06 16:53:57,489 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000785
2017-02-06 16:54:00,012 INFO    : Executor     : Collected FTrace binary trace:
2017-02-06 16:54:00,014 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/2/trace.dat
2017-02-06 16:54:00,015 INFO    : Executor     : Collected FTrace function profiling:
2017-02-06 16:54:00,016 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/2/trace_stat.json
2017-02-06 16:54:00,017 INFO    : Executor     : Un-freezing userspace tasks
2017-02-06 16:54:00,330 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-06 16:54:00,332 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-06 16:54:00,333 INFO    : Executor     : Experiment 2/10, [myconf:mywl] 3/10
2017-02-06 16:54:00,369 INFO    : Executor     : Freezing all tasks except: sh,adbd,init,usb,transport,thermal-engine
2017-02-06 16:54:02

2017-02-06 16:54:08,782 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000723
2017-02-06 16:54:11,321 INFO    : Executor     : Collected FTrace binary trace:
2017-02-06 16:54:11,323 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/3/trace.dat
2017-02-06 16:54:11,324 INFO    : Executor     : Collected FTrace function profiling:
2017-02-06 16:54:11,325 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/3/trace_stat.json
2017-02-06 16:54:11,326 INFO    : Executor     : Un-freezing userspace tasks
2017-02-06 16:54:11,652 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-06 16:54:11,654 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-06 16:54:11,655 INFO    : Executor     : Experiment 3/10, [myconf:mywl] 4/10
2017-02-06 16:54:11,713 INFO    : Executor     : Freezing all tasks except: sh,adbd,init,usb,transport,thermal-engine
2017-02-06 16:54:13

2017-02-06 16:54:20,124 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000713
2017-02-06 16:54:22,674 INFO    : Executor     : Collected FTrace binary trace:
2017-02-06 16:54:22,676 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/4/trace.dat
2017-02-06 16:54:22,677 INFO    : Executor     : Collected FTrace function profiling:
2017-02-06 16:54:22,678 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/4/trace_stat.json
2017-02-06 16:54:22,679 INFO    : Executor     : Un-freezing userspace tasks
2017-02-06 16:54:22,990 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-06 16:54:22,992 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-06 16:54:22,993 INFO    : Executor     : Experiment 4/10, [myconf:mywl] 5/10
2017-02-06 16:54:23,062 INFO    : Executor     : Freezing all tasks except: sh,adbd,init,usb,transport,thermal-engine
2017-02-06 16:54:24

...

2017-02-06 16:54:42,846 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000711
2017-02-06 16:54:45,230 INFO    : Executor     : Collected FTrace binary trace:
2017-02-06 16:54:45,232 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/6/trace.dat
2017-02-06 16:54:45,233 INFO    : Executor     : Collected FTrace function profiling:
2017-02-06 16:54:45,235 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/6/trace_stat.json
2017-02-06 16:54:45,236 INFO    : Executor     : Un-freezing userspace tasks
2017-02-06 16:54:45,514 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-06 16:54:45,517 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-06 16:54:45,518 INFO    : Executor     : Experiment 6/10, [myconf:mywl] 7/10
2017-02-06 16:54:45,570 INFO    : Executor     : Freezing all tasks except: sh,adbd,init,usb,transport,thermal-engine
2017-02-06 16:54:47

2017-02-06 16:54:53,832 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000672
2017-02-06 16:54:56,056 INFO    : Executor     : Collected FTrace binary trace:
2017-02-06 16:54:56,058 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/7/trace.dat
2017-02-06 16:54:56,059 INFO    : Executor     : Collected FTrace function profiling:
2017-02-06 16:54:56,060 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/7/trace_stat.json
2017-02-06 16:54:56,061 INFO    : Executor     : Un-freezing userspace tasks
2017-02-06 16:54:56,328 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-06 16:54:56,331 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-06 16:54:56,332 INFO    : Executor     : Experiment 7/10, [myconf:mywl] 8/10
2017-02-06 16:54:56,401 INFO    : Executor     : Freezing all tasks except: sh,adbd,init,usb,transport,thermal-engine
2017-02-06 16:54:58

2017-02-06 16:55:04,840 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000738
2017-02-06 16:55:07,287 INFO    : Executor     : Collected FTrace binary trace:
2017-02-06 16:55:07,289 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/8/trace.dat
2017-02-06 16:55:07,290 INFO    : Executor     : Collected FTrace function profiling:
2017-02-06 16:55:07,291 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/8/trace_stat.json
2017-02-06 16:55:07,292 INFO    : Executor     : Un-freezing userspace tasks
2017-02-06 16:55:07,611 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-06 16:55:07,613 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-06 16:55:07,614 INFO    : Executor     : Experiment 8/10, [myconf:mywl] 9/10
2017-02-06 16:55:07,652 INFO    : Executor     : Freezing all tasks except: sh,adbd,init,usb,transport,thermal-engine
2017-02-06 16:55:09

2017-02-06 16:55:16,111 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000704
2017-02-06 16:55:18,632 INFO    : Executor     : Collected FTrace binary trace:
2017-02-06 16:55:18,634 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/9/trace.dat
2017-02-06 16:55:18,635 INFO    : Executor     : Collected FTrace function profiling:
2017-02-06 16:55:18,637 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/9/trace_stat.json
2017-02-06 16:55:18,638 INFO    : Executor     : Un-freezing userspace tasks
2017-02-06 16:55:18,974 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-06 16:55:18,976 INFO    : Executor     : ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
2017-02-06 16:55:18,978 INFO    : Executor     : Experiment 9/10, [myconf:mywl] 10/10
2017-02-06 16:55:19,016 INFO    : Executor     : Freezing all tasks except: sh,adbd,init,usb,transport,thermal-engine
2017-02-06 16:55:2

2017-02-06 16:55:27,383 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.000715
2017-02-06 16:55:29,852 INFO    : Executor     : Collected FTrace binary trace:
2017-02-06 16:55:29,854 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/10/trace.dat
2017-02-06 16:55:29,855 INFO    : Executor     : Collected FTrace function profiling:
2017-02-06 16:55:29,856 INFO    : Executor     :    <res_dir>/rtapp:myconf:mywl/10/trace_stat.json
2017-02-06 16:55:29,857 INFO    : Executor     : Un-freezing userspace tasks
2017-02-06 16:55:30,171 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-06 16:55:30,173 INFO    : Executor     : Restoring target SELinux mode: Permissive

2017-02-06 16:55:30,252 INFO    : Executor     : 
2017-02-06 16:55:30,254 INFO    : Executor     : ################################################################################
2017-02-06 16:55:30,255 INFO    : Executor     : Experiments execution complet

In [30]:
for experiment in executor.experiments:
    plot_trace(Trace(te.platform, experiment.out_dir, ['sched_switch']).ftrace)

In [13]:
def examine_experiment(experiment):
    trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    ma = SchedMultiAssert(trace.ftrace, hikey_topology, experiment.wload.tasks.keys())
    cluster_conclusions = []
    for cluster in te.topology.get_level('cluster'):
        residencies = [t['residency'] for p, t in ma.getResidency('cluster', cluster, percent=True).iteritems()]
        if all(r > 90 for r in residencies):
            cluster_conclusions.append('PACKED')
        else:
            cluster_conclusions.append(None)
    packed = [i for i, c in enumerate(cluster_conclusions) if c == 'PACKED']
    with open(os.path.join(experiment.out_dir, 'energy.json')) as f:
        energy = json.load(f)
        energy = energy['BAT']
    if not packed:
        print "I don't think we packed onto any cluster"
        return False, energy
        # plot_trace(trace.ftrace)
    elif len(packed) == 1:
        [i] = packed
        print 'I think we packed onto cluster {} ({})'.format(i, te.topology.get_node('cluster', i))
        return True, energy
    else:
        # this code is
        print 'what is this'
        # who even.. who wrote this shit
        print 'i dont even'
        # Microsoft recommends Windows 10 with Microsoft Edge® for a more secure browsing experience
        raise Exception(RuntimeError(GenericBamboozlementComplaintFactory2().generateComplaint('bamboozled!!!')))

In [22]:
previous_energy_records = [[], []]

In [23]:
energy_records = previous_energy_records
for experiment in executor.experiments:
    packed, energy = examine_experiment(experiment)
    energy_records[packed].append(energy)

I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster
I don't think we packed onto any cluster


In [16]:
import numpy as np

In [24]:
np.mean(energy_records[0]), np.std(energy_records[0])

(0.00071708485393796547, 2.7856461802274161e-05)

In [25]:
np.mean(energy_records[1]), np.std(energy_records[1])

/home/brendan/.local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/brendan/.local/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/brendan/.local/lib/python2.7/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/brendan/.local/lib/python2.7/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/brendan/.local/lib/python2.7/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(nan, nan)

In [19]:
energy_records

[[0.000678231252576252,
  0.000706550741657287,
  0.0006943539062684752,
  0.0006897877310203882,
  0.0007237108684665963,
  0.0006976488512009929,
  0.0006631306221278609,
  0.0006860670058593412,
  0.0007028870060453296,
  0.0007852204391305962,
  0.0007234033282422243,
  0.0007132371062019097,
  0.0007058232629383376,
  0.0007112573068975314,
  0.0006716430981954398,
  0.0007384731403419589,
  0.0007039138473656904,
  0.0007149900040206378],
 [0.000687672569570034, 0.0006595024601106137]]

In [20]:
raise Exception('HALT HALT HALT, HALT AT THE GATES OF THE GRAVEYARD OR SURELY PERISH')

Exception: HALT HALT HALT, HALT AT THE GATES OF THE GRAVEYARD OR SURELY PERISH

##### 💀💀 The code graveyard 💀💀
*abandon hope all ye who scroll past here, for here is the threshold of the graveyard, whither is sent the code of yore, remembered but not missed, respected but not loved, written by the hand of fearful men and trialled by the silicon of scorned machines*

In [ ]:
def get_task_distribs(nrg_model, cpu=0, task_pct=4):
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    ncpus = len(cluster_node.cpus)
    min_cap = cpu_node.min_capacity
    
    task_cap = int(1024 * (task_pct / 100.))
    
    tasks_per_cpu = int(min_cap / task_cap)
    task_distribs = get_various_packing_placements(tasks_per_cpu * ncpus, ncpus)
    return task_distribs

In [ ]:
def get_util_distribs(nrg_model, task_distribs=None, cpu=0, task_pct=4):
    if task_distribs == None:
        task_distribs = get_task_distribs(nrg_model, cpu, task_pct)
    util_distribs = []
    
    task_cap = int(1024 * (task_pct / 100.))
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    
    for task_distrib in task_distribs:
        util_distrib = [0 for _ in range(len(nrg_model.cpus))]
        for i, num_tasks_on_cpu in enumerate(task_distrib):
            util_distrib[cluster_node.cpus[i]] = num_tasks_on_cpu * task_cap
        util_distribs.append(util_distrib)
    return util_distribs

In [ ]:
def get_wloads(nrg_model, task_distribs=None, cpu=0, task_pct=4):
    if task_distribs == None:
        task_distribs = get_task_distribs(nrg_model, cpu, task_pct)
    for task_distrib in 

In [ ]:
util_distribs = get_util_distribs(hikey_energy)
for ud in util_distribs:
    display(hikey_energy.estimate_from_cpu_util(ud))

In [ ]:
hikey_energy.cpu_nodes[0].min_capacity

In [ ]:
def get_cluster_count_df(experiment, clusters, trace=None):
    if trace is None:
        trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    tasks = experiment.wload.tasks.keys()
    pids = [trace.getTaskByName(t)[0] for t in tasks]
    task_cpus = trace.data_frame.task_cpus()[pids]
    # drop consecutive duplicates
    task_cpus = task_cpus[(task_cpus.shift(+1) != task_cpus).any(axis=1)]

    df = pd.DataFrame(columns=[str(c) for c in clusters])
    
    for (time, row_in) in task_cpus.iterrows():
        cluster_task_counts = [0 for c in clusters]
        for task_cpu in row_in:
            if isnan(task_cpu):
                continue
            for i, cluster in enumerate(clusters):
                if int(task_cpu) in cluster:
                    cluster_task_counts[i] += 1
                    break
    
        df.loc[time] = pd.Series(cluster_task_counts, index=df.columns)
        
    return df

def were_tasks_cluster_packed(experiment, clusters, trace=None):
    if trace is None:
        trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    cluster_counts = get_cluster_count_df(experiment, clusters, trace)
    

In [ ]:
def get_nrg(model, util):
    nrg = model.estimate_from_cpu_util(util, zero_idle=True)
    return sum(nrg.values())